In [ ]:
from datetime import datetime
from packaging import version
import os
import sys
sys.path.append('./metalearning')
try:
  from google.colab import drive
  drive.mount('/content/drive')
  inColab = True
except:
  pass
os.environ["DatabaseDir"]="/content/drive/My Drive/Thesis/Shared/Datasets/"
os.environ["DATASRC"]="/content/datasets"
os.environ["SPLITS"]="/content/datasets/Splits"
os.environ["RECORDS"]="/content/datasets/Records"
if not os.path.isdir(os.environ["DATASRC"]):
  os.mkdir(os.environ["DATASRC"])

Mounted at /content/drive


In [ ]:
if not os.path.exists(os.path.join(os.environ["DATASRC"],"Records")):
  !cp /content/drive/My\ Drive/Thesis/Shared/Datasets/records.zip "$DATASRC"
  !cp /content/drive/My\ Drive/Thesis/Shared/Datasets/splits.zip "$DATASRC"

In [ ]:
if not os.path.exists(os.path.join(os.environ["DATASRC"],"Records")):
  os.chdir(os.environ["DATASRC"])
  !unzip -q splits.zip  -d ./
  !unzip -q records.zip  -d ./

In [ ]:
%load_ext tensorboard

In [ ]:
sys.path.append('/content/metalearning')


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from packaging import version
import os
import sys

from metalearning_tasks.fixed_shot_classification_v2 import DatasetGenerator
from models.model_builder import create_one_hot_model_convolutional, create_cnn, create_cell

In [ ]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_error = tf.keras.metrics.Mean('train_error', dtype=tf.float32)

In [ ]:
exp_no = 18
batch_size = _batch_size = 32
nb_classes = _nb_classes = 5
nb_samples_per_class = _nb_samples_per_class = 10
_input_height = _input_width = 112
colorspace = 'RGB'
channels = {'RGB':3, 'L':1}
_iterations = 100000

img_size = (_input_height, _input_width)
img_shape = [_input_height, _input_width, channels[colorspace]]

input_size = _input_height * _input_width * channels[colorspace]
cell = 'DNC'
nb_reads = _nb_reads = 1
controller_size = _controller_size = 100
memory_size = _memory_locations = 128
memory_dim = ar_memory_word_size = 20
summary_interval = 100
checkpt_write_interval = 1000
dataset = 'vgg_flower'
splits=[70,15,15]

learning_rate = _learning_rate = 1e-4
start=0
save_dir='/content/drive/MyDrive/Documents/MSC/Courses/Thesis/Experiments_New/Convolutional4L'
_start_iterations = 0

In [ ]:
%tensorboard --logdir '{save_dir}'

<IPython.core.display.Javascript object>

In [ ]:
def metric_accuracy(_nb_classes,_nb_samples_per_class,labels, outputs):
    seq_length = _nb_classes * _nb_samples_per_class
    outputs = np.argmax(outputs, axis=-1)
    correct = [0] * seq_length
    total = [0] * seq_length
    for i in range(np.shape(labels)[0]):
        label = labels[i]
        output = outputs[i]
        class_count = {}
        for j in range(seq_length):
            class_count[label[j]] = class_count.get(label[j], 0) + 1
            total[class_count[label[j]]] += 1
            if label[j] == output[j]:
                correct[class_count[label[j]]] += 1
    return [float(correct[i]) / total[i] if total[i] > 0. else 0.
            for i in range(1, _nb_samples_per_class + 1)]
last_logged_ep=0

@tf.function
def trainstep(model, data_generator, optimizer):
    image, label = data_generator.generate_batch("train",
                                                 _batch_size)
    images = tf.reshape(image, (_batch_size,-1,_input_width,_input_height,3))
    one_hot_target = tf.one_hot(label, _nb_classes, axis=-1)
    offset_target_var = tf.concat([tf.zeros_like(tf.expand_dims(
        one_hot_target[:, 0], 1)), one_hot_target[:, :-1]], axis=1)

    with tf.GradientTape() as tape:
        output = model([images, offset_target_var])
        loss = tf.reduce_mean(
                    tf.reduce_sum(
                        tf.nn.softmax_cross_entropy_with_logits(
                            labels=one_hot_target,
                            logits=output
                        ),
                        axis=1)
                  )
        grads, _ = tf.clip_by_global_norm(
            tape.gradient(loss, model.trainable_variables),
            50)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

def train(model, data_genarator, _start_iterations, _iterations, _learning_rate, cell, _batch_size, _nb_classes, save_dir):
    optimizer = tf.keras.optimizers.Adam(lr=_learning_rate)
    print("1st\t2nd\t3rd\t4th\t5th\t6th\t7th\t8th\t9th\t10th\tbatch\tloss\tdt")
    logdir = f"{save_dir}/{dataset}/{cell}/{_learning_rate}/{exp_no}/logs"
    summary_writer = tf.summary.create_file_writer(logdir)
    dt = datetime.now()
    accuracies = []
    losses = []
    for ep in range(_start_iterations, _iterations):
        if ep % summary_interval == 0:
            image, label = data_generator.generate_batch("train",
                                                         _batch_size)
            images = tf.reshape(image, (_batch_size,-1,_input_width,_input_height,3))
            one_hot_target = tf.one_hot(label, _nb_classes, axis=-1)
            offset_target_var = tf.concat([tf.zeros_like(tf.expand_dims(
                one_hot_target[:, 0], 1)), one_hot_target[:, :-1]], axis=1)
            output = model([images, offset_target_var])
            loss = tf.reduce_mean(
                        tf.reduce_sum(
                            tf.nn.softmax_cross_entropy_with_logits(
                                labels=one_hot_target,
                                logits=output),
                            axis=1))
            accuracy = metric_accuracy(_nb_classes, _nb_samples_per_class, label, output)
            accuracies.append((ep,accuracy))
            dt = datetime.now() - dt

            for accu in accuracy:
                print('%.4f' % accu, end='\t')
            print('%d\t%.4f\t%.4f' % (ep, loss, dt.total_seconds()))
            train_loss(loss)
            losses.append((ep,train_loss.result()))

            dt = datetime.now()
        if ep % checkpt_write_interval == 1 and ep > 0:
            model.save_weights(f"{save_dir}/{dataset}/{cell}/{_learning_rate}/{exp_no}" + "/model.")
            with open(f"{save_dir}/{dataset}/{cell}/{_learning_rate}/{exp_no}" + "/model.iteration", 'w') as f:
                _start_iterations =  f.write(str(ep))
            with summary_writer.as_default():
              for ep_idx, accuracy in accuracies:
                for i, accu in enumerate(accuracy):
                    tf.summary.scalar(f'train_acc_{i}',
                                      accu,
                                      step=ep_idx)
              for ep_idx, loss_res in losses:
                tf.summary.scalar('train_loss',
                                  train_loss.result(),
                                  step=ep_idx)
            last_logged_ep=ep
            accuracies=[]
            losses=[]
        loss = trainstep(model, data_generator, optimizer)
        train_loss(loss)


In [ ]:
def create_one_hot_model_convolutional(image_size,
                                       extra_channel,
                                       output_size,
                                       batch_size,
                                       cell,
                                       cnn):
    labels = tf.keras.layers.Input([None] + extra_channel,
                                   batch_size=batch_size)
    images = tf.keras.layers.Input([None] + image_size,
                                   batch_size=batch_size)
    labels0 = tf.stop_gradient(labels)
    images0 = tf.stop_gradient(images)
    resnet = tf.keras.applications.EfficientNetB0(
                                                    include_top=False, weights='imagenet',
                                                    input_shape=(_input_width,_input_height,3)
                                                  )
    cnn_timeseries = tf.keras.layers.TimeDistributed(resnet, trainable=False)
    flatten = tf.keras.layers.Flatten()
    flatten_timeseries = tf.keras.layers.TimeDistributed(flatten)
    dropout = tf.keras.layers.Dropout(rate=0.4)
    dropout_timeseries = tf.keras.layers.TimeDistributed(dropout)
    linear = tf.keras.layers.Dense(1600)
    linear_timeseries = tf.keras.layers.TimeDistributed(linear)
    cell = create_cell(cell,
                       output_size,
                       100,
                       128,
                       20,
                       10)

    initial_state = cell.get_initial_state(batch_size=batch_size,
                                           dtype=tf.float32)
    rnn = tf.keras.layers.RNN(cell, return_sequences=True)
    cnn_out = cnn_timeseries(images0)
    flatten_out = flatten_timeseries(cnn_out)
    dropout_out = dropout_timeseries(flatten_out, training=True)
    linear_out = linear_timeseries(dropout_out)
    cnn_out_plus_labels = tf.concat([linear_out, labels0], axis=-1)
    y1 = rnn(cnn_out_plus_labels, initial_state=initial_state)
    y2 = tf.keras.layers.Dense(output_size)(y1)
    y3 = tf.keras.layers.Softmax()(y2)
    model = tf.keras.Model([images, labels], y3)
    return model

In [ ]:
model = create_one_hot_model_convolutional(img_shape,
                                           [5],
                                           5,
                                           _batch_size,
                                           cell,
                                          '4layerConv')


16711680/16705208 [==============================] - 0s 0us/step


In [ ]:
ds_root = f'/content/datasets/Records/{dataset}'
data_generator = DatasetGenerator(data_folder=ds_root,
                                  splits=splits,
                                  nb_samples_per_class=nb_samples_per_class,
                                  img_size=img_size,
                                  colorspace=colorspace,
                                  pre_scale=(_input_width,_input_height),
                                  augment=False
                                )

In [ ]:
model.summary()

In [ ]:
model.load_weights(f"{save_dir}/{dataset}/{cell}/{learning_rate}/{exp_no}" + "/model.")

In [ ]:
b1 = data_generator.generate_batch('train', _batch_size)

In [ ]:
plt.imshow(tf.reshape(b1[0][0,0,:],(_input_width,_input_height,3)))

In [ ]:
train(model, data_generator, _start_iterations, _iterations, _learning_rate, cell, _batch_size, _nb_classes, save_dir)

In [ ]:
def create_4layer_conv(image_size):
    in_image = tf.keras.layers.Input(image_size)
    x = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')(in_image)
    x = tf.keras.layers.BatchNormalization()(x)
    for i in range(3):
        with tf.name_scope(f"convbn_layer{i}"):
            x = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')(x)
            x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Flatten()(x)
    cnn_model = tf.keras.Model(in_image, x)
    return cnn_model

In [ ]:
def create_4layer_deconv(image_size):
    in_image = tf.keras.layers.Input(1600)
    x = tf.keras.layers.Dense(np.prod([40, 40, 64]))(in_image)
    x = tf.reshape(x,(40, 40, 64))
    x = tf.keras.layers.Conv2DTranspose(64, (3, 3), padding='SAME')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    for i in range(3):
        with tf.name_scope(f"convbn_layer{i}"):
            x = tf.keras.layers.Conv2DTranspose(64, (3, 3), padding='SAME')(x)
            x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    cnn_model = tf.keras.Model(in_image, x)
    return cnn_model

In [ ]:
encoder = create_4layer_conv((40,40,3))
bottleneck = tf.keras.layers.Dense(1600)
decoder = create_4layer_deconv((40,40,3))

In [ ]:
cnn.summary()

In [ ]:
!cp -r ./Experiments_New/Convolutional4L/ /content/drive/MyDrive/Documents/MSC/Courses/Thesis/Experiments_New/